<a href="https://colab.research.google.com/github/ragaja-14/spark_1/blob/master/Assignment_4_202018012.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark
import pyspark

     |████████████████████████████████| 204.2MB 72kB/s 
     |████████████████████████████████| 204kB 48.5MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612242 sha256=c8b8876400c5b5874508c7bfd68381028981b202d7285b9117a05d0fc19401ae
  Stored in directory: /root/.cache/pip/wheels/5e/bd/07/031766ca628adec8435bb40f0bd83bb676ce65ff4007f8e73f
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('Assignment 4').getOrCreate()

In [ ]:
df = spark.read.csv('Car details v3.csv',inferSchema=True,header=True)

In [ ]:
df.printSchema()
df.count()

root
 |-- name: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- selling_price: integer (nullable = true)
 |-- km_driven: integer (nullable = true)
 |-- fuel: string (nullable = true)
 |-- seller_type: string (nullable = true)
 |-- transmission: string (nullable = true)
 |-- owner: string (nullable = true)
 |-- mileage: string (nullable = true)
 |-- engine: string (nullable = true)
 |-- max_power: string (nullable = true)
 |-- torque: string (nullable = true)
 |-- seats: integer (nullable = true)



8128

In [ ]:
df.show(n=10)

+--------------------+----+-------------+---------+------+-----------+------------+------------+----------+-------+----------+--------------------+-----+
|                name|year|selling_price|km_driven|  fuel|seller_type|transmission|       owner|   mileage| engine| max_power|              torque|seats|
+--------------------+----+-------------+---------+------+-----------+------------+------------+----------+-------+----------+--------------------+-----+
|Maruti Swift Dzir...|2014|       450000|   145500|Diesel| Individual|      Manual| First Owner| 23.4 kmpl|1248 CC|    74 bhp|      190Nm@ 2000rpm|    5|
|Skoda Rapid 1.5 T...|2014|       370000|   120000|Diesel| Individual|      Manual|Second Owner|21.14 kmpl|1498 CC|103.52 bhp| 250Nm@ 1500-2500rpm|    5|
|Honda City 2017-2...|2006|       158000|   140000|Petrol| Individual|      Manual| Third Owner| 17.7 kmpl|1497 CC|    78 bhp|12.7@ 2,700(kgm@ ...|    5|
|Hyundai i20 Sport...|2010|       225000|   127000|Diesel| Individual|      

Conversion of max_power, mileage, engine columns to numeric values from string 

In [ ]:
from pyspark.sql.functions import regexp_replace, col,split
from pyspark.sql.types import DoubleType, IntegerType, FloatType


df=df.withColumn("name",split(df['name'], ' ').getItem(0))
df=df.withColumn("max_power",regexp_replace(col("max_power"), "bhp", "").cast(FloatType()))
df=df.withColumn("mileage",regexp_replace(col("mileage"), "kmpl|km/kg", "").cast(FloatType()))
df=df.withColumn("engine",regexp_replace(col("engine"), "CC", "").cast(IntegerType()))


In [ ]:
df=df.dropna()
df.show(10)

+-------+----+-------------+---------+------+-----------+------------+------------+-------+------+---------+--------------------+-----+
|   name|year|selling_price|km_driven|  fuel|seller_type|transmission|       owner|mileage|engine|max_power|              torque|seats|
+-------+----+-------------+---------+------+-----------+------------+------------+-------+------+---------+--------------------+-----+
| Maruti|2014|       450000|   145500|Diesel| Individual|      Manual| First Owner|   23.4|  1248|     74.0|      190Nm@ 2000rpm|    5|
|  Skoda|2014|       370000|   120000|Diesel| Individual|      Manual|Second Owner|  21.14|  1498|   103.52| 250Nm@ 1500-2500rpm|    5|
|  Honda|2006|       158000|   140000|Petrol| Individual|      Manual| Third Owner|   17.7|  1497|     78.0|12.7@ 2,700(kgm@ ...|    5|
|Hyundai|2010|       225000|   127000|Diesel| Individual|      Manual| First Owner|   23.0|  1396|     90.0|22.4 kgm at 1750-...|    5|
| Maruti|2007|       130000|   120000|Petrol| In

**Case 1**
**Using one hot encoding and vector assembler**

**Polynomial features for regression and**

**MinMaxScaler for normalization of the features**

**No feature hasher is used**

**And Building Regression model and evaluating with rmse, mse, r2**

Use One Hot Encoding API to encode the categorical features.

In [ ]:
from pyspark.ml.feature import StringIndexer , OneHotEncoder

cat_features=['name','fuel','seller_type','transmission','owner']

indexer1=StringIndexer(inputCols=cat_features,outputCols=['indexed_'+i for i in cat_features])

indexed_df=indexer1.fit(df).transform(df)
print(indexed_df.count())

7906


Use One Hot Encoding API to encode the categorical features

In [ ]:
cat_features=['name','fuel','seller_type','transmission','owner']
for i in cat_features:
  print("Count of ",i,indexed_df.select(i).distinct().count())

Count of  name 31
Count of  fuel 4
Count of  seller_type 3
Count of  transmission 2
Count of  owner 5


In [ ]:
indexed_features=['indexed_'+i for i in cat_features[1:]]

myencoder=OneHotEncoder(inputCols=indexed_features,outputCols=['Encoded_'+i for i in cat_features[1:]])

encoded_df=myencoder.fit(indexed_df).transform(indexed_df)
encoded_df.show()

+--------+----+-------------+---------+------+-----------+------------+------------+-------+------+---------+--------------------+-----+------------+------------+--------------------+-------------------+-------------+-------------+-------------------+--------------------+-------------+
|    name|year|selling_price|km_driven|  fuel|seller_type|transmission|       owner|mileage|engine|max_power|              torque|seats|indexed_fuel|indexed_name|indexed_transmission|indexed_seller_type|indexed_owner| Encoded_fuel|Encoded_seller_type|Encoded_transmission|Encoded_owner|
+--------+----+-------------+---------+------+-----------+------------+------------+-------+------+---------+--------------------+-----+------------+------------+--------------------+-------------------+-------------+-------------+-------------------+--------------------+-------------+
|  Maruti|2014|       450000|   145500|Diesel| Individual|      Manual| First Owner|   23.4|  1248|     74.0|      190Nm@ 2000rpm|    5|   

In [ ]:
encoded_df.columns

['name',
 'year',
 'selling_price',
 'km_driven',
 'fuel',
 'seller_type',
 'transmission',
 'owner',
 'mileage',
 'engine',
 'max_power',
 'torque',
 'seats',
 'indexed_fuel',
 'indexed_name',
 'indexed_transmission',
 'indexed_seller_type',
 'indexed_owner',
 'Encoded_fuel',
 'Encoded_seller_type',
 'Encoded_transmission',
 'Encoded_owner']

In [ ]:
##Using vector assembler to combine all required feature into single vector form
from pyspark.ml.feature import VectorAssembler

#using the numeric columns and newly encoded columns to be combined to form a single vector form
input_features=['seats',
 'km_driven',
 'year',
 'mileage',
 'engine',
 'max_power',
 'indexed_name',
 'Encoded_fuel',
 'Encoded_seller_type',
 'Encoded_transmission',
 'Encoded_owner']

assembler =VectorAssembler(inputCols=input_features ,outputCol="features1")
new_df=assembler.transform(encoded_df)
vectorized_df=new_df.select("features1","selling_price")
vectorized_df.show(n=10)

+--------------------+-------------+
|           features1|selling_price|
+--------------------+-------------+
|(17,[0,1,2,3,4,5,...|       450000|
|[5.0,120000.0,201...|       370000|
|[5.0,140000.0,200...|       158000|
|[5.0,127000.0,201...|       225000|
|(17,[0,1,2,3,4,5,...|       130000|
|[5.0,45000.0,2017...|       440000|
|(17,[0,1,2,3,4,5,...|        96000|
|(17,[0,1,2,3,4,5,...|        45000|
|[5.0,90000.0,2011...|       350000|
|[5.0,169000.0,201...|       200000|
+--------------------+-------------+
only showing top 10 rows



> Using degree: 4 for polynomial expansion

In [ ]:
from pyspark.ml.feature import PolynomialExpansion

polyExpansion = PolynomialExpansion(degree=4, inputCol="features1", outputCol="polyFeatures")
polyDF = polyExpansion.transform(vectorized_df)
print("Using Vectorized features:")
polyDF.show(n=10)

Using Vectorized features:
+--------------------+-------------+--------------------+
|           features1|selling_price|        polyFeatures|
+--------------------+-------------+--------------------+
|(17,[0,1,2,3,4,5,...|       450000|(5984,[0,1,2,3,4,...|
|[5.0,120000.0,201...|       370000|[5.0,25.0,125.0,6...|
|[5.0,140000.0,200...|       158000|[5.0,25.0,125.0,6...|
|[5.0,127000.0,201...|       225000|[5.0,25.0,125.0,6...|
|(17,[0,1,2,3,4,5,...|       130000|(5984,[0,1,2,3,4,...|
|[5.0,45000.0,2017...|       440000|[5.0,25.0,125.0,6...|
|(17,[0,1,2,3,4,5,...|        96000|(5984,[0,1,2,3,4,...|
|(17,[0,1,2,3,4,5,...|        45000|(5984,[0,1,2,3,4,...|
|[5.0,90000.0,2011...|       350000|[5.0,25.0,125.0,6...|
|[5.0,169000.0,201...|       200000|[5.0,25.0,125.0,6...|
+--------------------+-------------+--------------------+
only showing top 10 rows



In [ ]:
from pyspark.ml.feature import MinMaxScaler

scaler=MinMaxScaler(inputCol="polyFeatures", outputCol="scaledFeatures")
scaler_df = scaler.fit(polyDF).transform(polyDF)

scaler_df.select("polyFeatures", "scaledFeatures").show(n=5)

+--------------------+--------------------+
|        polyFeatures|      scaledFeatures|
+--------------------+--------------------+
|(5984,[0,1,2,3,4,...|(5984,[0,1,2,3,4,...|
|[5.0,25.0,125.0,6...|(5984,[0,1,2,3,4,...|
|[5.0,25.0,125.0,6...|(5984,[0,1,2,3,4,...|
|[5.0,25.0,125.0,6...|(5984,[0,1,2,3,4,...|
|(5984,[0,1,2,3,4,...|(5984,[0,1,2,3,4,...|
+--------------------+--------------------+
only showing top 5 rows



In [ ]:
from pyspark.ml.regression import LinearRegression 

train_data,test_data = scaler_df.randomSplit([0.8,0.2],seed=2)
Regressor=LinearRegression(featuresCol ='scaledFeatures', labelCol ='selling_price') 
Regressor=Regressor.fit(train_data)

predicited=Regressor.evaluate(test_data)

coeff = Regressor.coefficients

#X and Y intercept
intr = Regressor.intercept

print("Polynomial Regression using degree : 4")
print ("The coefficient of the model is : %a" %coeff)
print ("The Intercept of the model is : %f" %intr)

Polynomial Regression using degree : 4
The coefficient of the model is : DenseVector([-69935.2769, -28510.3241, -14057.7512, -19189.178, 208793.6429, 119540.2381, -2549.7101, -171472.3594, -4791.3244, -130348.8711, -302666.9648, -43942.0911, -116965.7931, 53962.6296, 265866.9822, -59445.5482, -20469.4759, -6067.5627, 204935.58, 120139.4768, 261.3453, -4973.4283, -129951.8998, -44331.2031, 272566.9609, -48731.2689, -12322.2115, 201244.4818, 120845.8181, -5134.7705, 279221.5113, -37802.4881, 197719.7548, 285830.9823, -4262.957, -10713.1526, 33065.8875, 57273.9567, -30313.191, -54498.5118, -120148.0618, -27559.8915, -146822.0738, -18267.6099, -2505.6519, -6049.4356, 38727.3177, -35790.0435, -55501.397, -27984.8035, -590.4359, -1278.5261, -41094.5574, 1478.8214, -24948.5094, 14427.7035, 72170.3497, 64601.3462, 78838.1963, 30255.2756, -24600.6962, 15873.0016, 59951.2521, -24170.3914, -156527.1744, -55770.085, 167189.4841, -156467.1127, -390998.2517, -55010.7762, -36905.0765, -38498.1008, -7

In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator


print("Polynomial Regression using degree : 4")
eval = RegressionEvaluator(labelCol="selling_price", predictionCol="prediction", metricName="rmse")

rmse = eval.evaluate(predicited.predictions)
print("RMSE: %.3f" %rmse)
mse = eval.evaluate(predicited.predictions, {eval.metricName: "mse"})
print("MSE: %.3f" % mse)

r2 = eval.evaluate(predicited.predictions, {eval.metricName: "r2"})
print("r2: %.3f" %r2)

Polynomial Regression using degree : 4
RMSE: 271780.004
MSE: 73864370456.921
r2: 0.901


> Using degree=2 for polynomial expansion

In [ ]:
polyExpansion2 = PolynomialExpansion(degree=2, inputCol="features1", outputCol="polyFeatures")
polyDF2 = polyExpansion2.transform(vectorized_df)
print("Using Vectorized features:")
polyDF2.show(n=10)


scaler2=MinMaxScaler(inputCol="polyFeatures", outputCol="scaledFeatures")
scaler_df2 = scaler2.fit(polyDF2).transform(polyDF2)
scaler_df2.select("polyFeatures", "scaledFeatures").show(n=5)


train_data2,test_data2 = scaler_df2.randomSplit([0.8,0.2],seed=2)
Regressor2=LinearRegression(featuresCol ='scaledFeatures', labelCol ='selling_price') 
Regressor2=Regressor2.fit(train_data2)

pred2=Regressor2.evaluate(test_data2)
coeff = Regressor2.coefficients
intr = Regressor2.intercept

print("Polynomial Regression using degree :2 ")
print ("The coefficient of the model is : %a" %coeff)
print ("The Intercept of the model is : %f" %intr)
eval = RegressionEvaluator(labelCol="selling_price", predictionCol="prediction", metricName="rmse")

rmse = eval.evaluate(pred2.predictions)
print("RMSE: %.3f" %rmse)
mse = eval.evaluate(pred2.predictions, {eval.metricName: "mse"})
print("MSE: %.3f" % mse)

r2 = eval.evaluate(pred2.predictions, {eval.metricName: "r2"})
print("r2: %.3f" %r2)

Using Vectorized features:
+--------------------+-------------+--------------------+
|           features1|selling_price|        polyFeatures|
+--------------------+-------------+--------------------+
|(17,[0,1,2,3,4,5,...|       450000|(170,[0,1,2,3,4,5...|
|[5.0,120000.0,201...|       370000|[5.0,25.0,120000....|
|[5.0,140000.0,200...|       158000|[5.0,25.0,140000....|
|[5.0,127000.0,201...|       225000|[5.0,25.0,127000....|
|(17,[0,1,2,3,4,5,...|       130000|(170,[0,1,2,3,4,5...|
|[5.0,45000.0,2017...|       440000|[5.0,25.0,45000.0...|
|(17,[0,1,2,3,4,5,...|        96000|(170,[0,1,2,3,4,5...|
|(17,[0,1,2,3,4,5,...|        45000|(170,[0,1,2,3,4,5...|
|[5.0,90000.0,2011...|       350000|[5.0,25.0,90000.0...|
|[5.0,169000.0,201...|       200000|[5.0,25.0,169000....|
+--------------------+-------------+--------------------+
only showing top 10 rows

+--------------------+--------------------+
|        polyFeatures|      scaledFeatures|
+--------------------+--------------------+
|(1

> On comparing the results using different degrees the higher degree of polynomial expansion by degree=4 gave better results than degree=2




---










**CASE 2**
**Using Feature hasher**

**Polynomial features for regression and**

**MinMaxScaler for normalization of the features** **bold text**

**And Building Regression model and evaluating with rmse, mse, r2**

Instead of explicit String indexing followed by one hot encoding the

Feature Hasher used implicitly handles the one hot encoding for categorical columns.

In [ ]:
from pyspark.ml.feature import FeatureHasher

ind_features=['year',
 'km_driven',
 'fuel',
 'seller_type',
 'transmission',
 'owner',
 'mileage',
 'engine',
 'max_power',
 'seats']
 #Reducing the 11 features set to dimension of 5  features
 #implicitly encodes catogerical columns by one hot encoding

hasher = FeatureHasher(numFeatures=5,inputCols=ind_features,
                       outputCol="features")
featurized = hasher.transform(df)
featurized.show(n=5,truncate=False)


+-------+----+-------------+---------+------+-----------+------------+------------+-------+------+---------+------------------------+-----+------------------------------------------------------+
|name   |year|selling_price|km_driven|fuel  |seller_type|transmission|owner       |mileage|engine|max_power|torque                  |seats|features                                              |
+-------+----+-------------+---------+------+-----------+------------+------------+-------+------+---------+------------------------+-----+------------------------------------------------------+
|Maruti |2014|450000       |145500   |Diesel|Individual |Manual      |First Owner |23.4   |1248  |74.0     |190Nm@ 2000rpm          |5    |(5,[1,2,3,4],[98.39999961853027,1248.0,6.0,147516.0]) |
|Skoda  |2014|370000       |120000   |Diesel|Individual |Manual      |Second Owner|21.14  |1498  |103.52   |250Nm@ 1500-2500rpm     |5    |(5,[1,2,3,4],[126.65999603271484,1498.0,6.0,122015.0])|
|Honda  |2006|158000     

In [ ]:
from pyspark.ml.feature import PolynomialExpansion

FH_polyExpansion = PolynomialExpansion(degree=4, inputCol="features", outputCol="polyFeatures")
FH_polyDF = FH_polyExpansion.transform(featurized.select('features','selling_price'))
print("On Using Feature Hasher and expanding the feature by degree=4 :")
FH_polyDF.show(n=10)

On Using Feature Hasher and expanding the feature by degree=4 :
+--------------------+-------------+--------------------+
|            features|selling_price|        polyFeatures|
+--------------------+-------------+--------------------+
|(5,[1,2,3,4],[98....|       450000|(125,[4,8,11,13,1...|
|(5,[1,2,3,4],[126...|       370000|(125,[4,8,11,13,1...|
|(5,[1,2,3,4],[95....|       158000|(125,[4,8,11,13,1...|
|(5,[1,2,3,4],[114...|       225000|(125,[4,8,11,13,1...|
|(5,[1,2,3,4],[104...|       130000|(125,[4,8,11,13,1...|
|(5,[1,2,3,4],[102...|       440000|(125,[4,8,11,13,1...|
|(5,[0,1,2,3,4],[1...|        96000|(125,[0,1,2,3,4,5...|
|(5,[1,2,3,4],[54....|        45000|(125,[4,8,11,13,1...|
|(5,[1,2,3,4],[91....|       350000|(125,[4,8,11,13,1...|
|(5,[1,2,3,4],[89....|       200000|(125,[4,8,11,13,1...|
+--------------------+-------------+--------------------+
only showing top 10 rows



In [ ]:
from pyspark.ml.feature import MinMaxScaler

scaler3=MinMaxScaler(inputCol="polyFeatures", outputCol="scaledFeatures")
FH_scaler_df = scaler3.fit(FH_polyDF).transform(FH_polyDF)

FH_scaler_df.select("polyFeatures", "scaledFeatures").show(n=5)

+--------------------+--------------------+
|        polyFeatures|      scaledFeatures|
+--------------------+--------------------+
|(125,[4,8,11,13,1...|(125,[4,8,11,13,1...|
|(125,[4,8,11,13,1...|(125,[4,8,11,13,1...|
|(125,[4,8,11,13,1...|(125,[4,8,11,13,1...|
|(125,[4,8,11,13,1...|(125,[4,8,11,13,1...|
|(125,[4,8,11,13,1...|(125,[4,8,11,13,1...|
+--------------------+--------------------+
only showing top 5 rows



In [ ]:
FH_train_data,FH_test_data = FH_scaler_df.randomSplit([0.8,0.2],seed=2)
FH_Regressor=LinearRegression(featuresCol ='scaledFeatures', labelCol ='selling_price') 
FH_Regressor=FH_Regressor.fit(FH_train_data)

FH_predicited=FH_Regressor.evaluate(FH_test_data)

coeff = FH_Regressor.coefficients

#X and Y intercept
intr = FH_Regressor.intercept

print("Using Feaure hasher to reduce the features space to dimension= 5 and \nPolynomial Regression using to expand by degree : 4 ")
print ("The coefficient of the model is : %a" %coeff)
print ("The Intercept of the model is : %f" %intr)

#Using evalaution mertics to determine the model performance
FH_eval = RegressionEvaluator(labelCol="selling_price", predictionCol="prediction", metricName="rmse")

rmse3 = FH_eval.evaluate(FH_predicited.predictions)
print("RMSE: %.3f" %rmse3)
mse3 = FH_eval.evaluate(FH_predicited.predictions, {FH_eval.metricName: "mse"})
print("MSE: %.3f" % mse3)

r2_FH = FH_eval.evaluate(FH_predicited.predictions, {FH_eval.metricName: "r2"})
print("r2: %.3f" %r2_FH)


Using Feaure hasher to reduce the features space to dimension= 5 and 
Polynomial Regression using to expand by degree : 4 
The coefficient of the model is : DenseVector([-55474.0117, -55474.0117, -55474.0117, -55474.0117, -1234593.1969, 3994.8349, 3994.8349, 3994.8349, 17387417.0118, -35597.8846, -35597.8846, 31576954.486, -231641.6507, -28859295.2001, -976636.3983, 28328.3112, 28328.3112, 28328.3112, 380178.3195, 14915.6121, 14915.6121, 7747395.5889, -193483.7041, 1304762.9696, 612785.9319, 61383.7402, 61383.7402, 36434.4733, -147540.7264, -3567641.362, 1804281.29, -101941.1901, -5813814.6124, 111081.7927, -681772.5723, -25831.1551, -25831.1551, -25831.1551, -3078072.0347, 48244.1905, 48244.1905, 5056678.0417, 23729.3172, 1295268.8563, -776271.0324, 70604.1188, 70604.1188, -1702238.7852, 73231.0012, -819837.8663, 301733.5197, 116627.2464, -2461386.5187, 1129899.7274, 1953459.613, -657.9792, -657.9792, -1145714.5808, 85786.5081, -1057441.8061, 321474.5343, 106312.5829, -1344656.936, 93

**CASE 3**

**Using one hot encoding and vector assembler**

**Polynomial features for regression and**

**Without Feature Scaling**

**And Building Regression model and evaluating with rmse, mse, r2**

In [ ]:
#Using string indexer and one hot encoder for catogerical columns

indexed_features=['indexed_'+i for i in cat_features[1:]]

myencoder=OneHotEncoder(inputCols=indexed_features,outputCols=['Encoded_'+i for i in cat_features[1:]])

encoded_df2=myencoder.fit(indexed_df).transform(indexed_df)
encoded_df2.show(n=10)

+-------+----+-------------+---------+------+-----------+------------+------------+-------+------+---------+--------------------+-----+------------+------------+--------------------+-------------------+-------------+-------------+-------------------+--------------------+-------------+
|   name|year|selling_price|km_driven|  fuel|seller_type|transmission|       owner|mileage|engine|max_power|              torque|seats|indexed_fuel|indexed_name|indexed_transmission|indexed_seller_type|indexed_owner| Encoded_fuel|Encoded_seller_type|Encoded_transmission|Encoded_owner|
+-------+----+-------------+---------+------+-----------+------------+------------+-------+------+---------+--------------------+-----+------------+------------+--------------------+-------------------+-------------+-------------+-------------------+--------------------+-------------+
| Maruti|2014|       450000|   145500|Diesel| Individual|      Manual| First Owner|   23.4|  1248|     74.0|      190Nm@ 2000rpm|    5|       

In [ ]:
from pyspark.ml.feature import VectorAssembler

#using the numeric columns and newly encoded columns to be combined to form a single vector form
input_features=['seats',
 'km_driven',
 'year',
 'mileage',
 'engine',
 'max_power',
 'indexed_name',
 'Encoded_fuel',
 'Encoded_seller_type',
 'Encoded_transmission',
 'Encoded_owner']

assembler =VectorAssembler(inputCols=input_features ,outputCol="features2")
new_df2=assembler.transform(encoded_df2)
vectorized_df2=new_df2.select("features2","selling_price")
vectorized_df2.show(n=10)

+--------------------+-------------+
|           features2|selling_price|
+--------------------+-------------+
|(17,[0,1,2,3,4,5,...|       450000|
|[5.0,120000.0,201...|       370000|
|[5.0,140000.0,200...|       158000|
|[5.0,127000.0,201...|       225000|
|(17,[0,1,2,3,4,5,...|       130000|
|[5.0,45000.0,2017...|       440000|
|(17,[0,1,2,3,4,5,...|        96000|
|(17,[0,1,2,3,4,5,...|        45000|
|[5.0,90000.0,2011...|       350000|
|[5.0,169000.0,201...|       200000|
+--------------------+-------------+
only showing top 10 rows



In [ ]:
from pyspark.ml.feature import PolynomialExpansion

polyExpansion2 = PolynomialExpansion(degree=4, inputCol="features2", outputCol="polyFeatures")
polyDF2= polyExpansion2.transform(vectorized_df2)
print("Using Vectorized features:")
polyDF2.show(n=10)

Using Vectorized features:
+--------------------+-------------+--------------------+
|           features2|selling_price|        polyFeatures|
+--------------------+-------------+--------------------+
|(17,[0,1,2,3,4,5,...|       450000|(5984,[0,1,2,3,4,...|
|[5.0,120000.0,201...|       370000|[5.0,25.0,125.0,6...|
|[5.0,140000.0,200...|       158000|[5.0,25.0,125.0,6...|
|[5.0,127000.0,201...|       225000|[5.0,25.0,125.0,6...|
|(17,[0,1,2,3,4,5,...|       130000|(5984,[0,1,2,3,4,...|
|[5.0,45000.0,2017...|       440000|[5.0,25.0,125.0,6...|
|(17,[0,1,2,3,4,5,...|        96000|(5984,[0,1,2,3,4,...|
|(17,[0,1,2,3,4,5,...|        45000|(5984,[0,1,2,3,4,...|
|[5.0,90000.0,2011...|       350000|[5.0,25.0,125.0,6...|
|[5.0,169000.0,201...|       200000|[5.0,25.0,125.0,6...|
+--------------------+-------------+--------------------+
only showing top 10 rows



In [ ]:
from pyspark.ml.regression import LinearRegression 

#using the features expanded by polynomial regression
train_data3,test_data3 = polyDF2.randomSplit([0.8,0.2],seed=2)
Regressor3=LinearRegression(featuresCol ='polyFeatures', labelCol ='selling_price') 
Regressor3=Regressor3.fit(train_data3)

pred3=Regressor3.evaluate(test_data3)

coeff = Regressor3.coefficients

#X and Y intercept
intr = Regressor3.intercept

print("Without feature scaling \nPolynomial Regression using degree : 4")
print ("The coefficient of the model is : %a" %coeff)
print ("The Intercept of the model is : %f" %intr)
eval3 = RegressionEvaluator(labelCol="selling_price", predictionCol="prediction", metricName="rmse")

rmse_mm = eval3.evaluate(pred3.predictions)
print("RMSE: %.3f" %rmse_mm )
mse_mm  = eval3.evaluate(pred3.predictions, {eval3.metricName: "mse"})
print("MSE: %.3f" % mse_mm)

r2_mm  = eval3.evaluate(pred3.predictions, {eval3.metricName: "r2"})
print("r2: %.3f" %r2_mm )

Without feature scaling 
Polynomial Regression using degree : 4
The coefficient of the model is : DenseVector([-5827.9419, -148.4894, -5.1377, -0.4997, 0.0885, 0.0101, -0.0, -0.0003, -0.0, -0.0, -0.0, -0.0, -0.0, 0.0, 10225.6823, -2.4664, -0.053, -0.0011, 0.0, 0.0, 0.0, -0.0, -0.0, -0.0, 2.6117, -0.001, -0.0, 0.0, 0.0, -0.0, 0.0009, -0.0, 0.0, 0.0, -101.4874, -56.9844, 15.7519, 1.9489, -0.0007, -0.0002, -0.0001, -0.0, -0.0, -0.0, -0.0296, -0.0159, 0.0092, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, 0.0, -14.1425, 2.0448, 2.0419, 0.0001, 0.0, 0.0, -0.0069, 0.0011, 0.0, -0.0, -2.1127, -0.1882, 0.0, -0.001, -0.1257, -18.46, -1.3636, -0.1035, -0.014, 0.0, 0.0, -0.0, -0.0, -0.0, -0.0, -0.0075, -0.0005, -0.0, 0.0, 0.0, -0.0, -0.0, -0.0, 0.0, -0.0, -0.4212, -0.0626, -0.0067, -0.0, -0.0, -0.0, -0.0001, -0.0, -0.0, 0.0, 0.0052, 0.002, -0.0, 0.0, -0.0021, -0.0004, 0.0, 0.0, 0.0, 0.0, -0.0, 0.0, 0.0, 0.0, 0.0, 0.0002, 0.0, -0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 393.295, 15.4633, -1.091, -0.3356, -

Based on result from CASE 3 and CASE 1 it can be observed that the much little higher error is observed in case when no feature scaling is used. But no major differences are observed.


**CASE 4**

**Using Imputer instead of dropna and using  Feature hasher**

**Polynomial features for regression and**

**MinMaxScaler for normalization of the features** **bold text**

**And Building Regression model and evaluating with rmse, mse, r2**

In [ ]:
df4 = spark.read.csv('Car details v3.csv',inferSchema=True,header=True)

df4=df4.withColumn("name",split(df4['name'], ' ').getItem(0))
df4=df4.withColumn("max_power",regexp_replace(col("max_power"), "bhp", "").cast(FloatType()))
df4=df4.withColumn("mileage",regexp_replace(col("mileage"), "kmpl|km/kg", "").cast(FloatType()))
df4=df4.withColumn("engine",regexp_replace(col("engine"), "CC", "").cast(IntegerType()))


In [ ]:
from pyspark.sql.functions import *
print("Null values count")
for i in df4.columns:
  print(i,":  ",df4.where(col(i).isNull(),).count())

Null values count
name :   0
year :   0
selling_price :   0
km_driven :   0
fuel :   0
seller_type :   0
transmission :   0
owner :   0
mileage :   221
engine :   221
max_power :   216
torque :   222
seats :   221


In [ ]:
from pyspark.ml.feature import Imputer

impute_cols=['seats','max_power','mileage','engine']
mean_imputer = Imputer(strategy='mean', inputCols=impute_cols,outputCols=['Imputed_'+i for i in impute_cols])
imputed_df=mean_imputer.fit(df4).transform(df4)
imputed_df.count()

8128

In [ ]:
from pyspark.ml.feature import PolynomialExpansion
from pyspark.ml.feature import FeatureHasher

imputed_cols=['name',
 'year',
 'selling_price',
 'km_driven',
 'fuel',
 'seller_type',
 'transmission',
 'owner',
 'Imputed_seats',
 'Imputed_max_power',
 'Imputed_mileage',
 'Imputed_engine']
 #Reducing the features set to dimension of 5  features
 #Feature hasher implicitly encodes catogerical columns by one hot encoding
hasher = FeatureHasher(numFeatures=5,inputCols=imputed_cols,
                       outputCol="features")
featurized = hasher.transform(imputed_df)
featurized.show(n=5,truncate=False)

IP_polyExpansion = PolynomialExpansion(degree=4, inputCol="features", outputCol="polyFeatures")
IP_polyDF = IP_polyExpansion.transform(featurized.select('features','selling_price'))
print("On Using Feature Hasher and expanding the feature by degree=4 :")
IP_polyDF.show(n=10)

+-------+----+-------------+---------+------+-----------+------------+------------+-------+------+---------+------------------------+-----+-------------+-----------------+---------------+--------------+--------------------------------------------------------------------------+
|name   |year|selling_price|km_driven|fuel  |seller_type|transmission|owner       |mileage|engine|max_power|torque                  |seats|Imputed_seats|Imputed_max_power|Imputed_mileage|Imputed_engine|features                                                                  |
+-------+----+-------------+---------+------+-----------+------------+------------+-------+------+---------+------------------------+-----+-------------+-----------------+---------------+--------------+--------------------------------------------------------------------------+
|Maruti |2014|450000       |145500   |Diesel|Individual |Manual      |First Owner |23.4   |1248  |74.0     |190Nm@ 2000rpm          |5    |5            |74.0         

In [ ]:
from pyspark.ml.feature import MinMaxScaler

scaler3=MinMaxScaler(inputCol="polyFeatures", outputCol="scaledFeatures")
IP_scaler_df = scaler3.fit(IP_polyDF).transform(IP_polyDF)

IP_scaler_df.select("polyFeatures", "scaledFeatures").show(n=5)

+--------------------+--------------------+
|        polyFeatures|      scaledFeatures|
+--------------------+--------------------+
|(125,[0,1,2,3,4,5...|[0.33333333333333...|
|(125,[0,1,2,3,4,5...|[0.33333333333333...|
|(125,[0,1,2,3,4,5...|[0.25,0.109375,0....|
|(125,[0,1,2,3,4,5...|[0.25,0.109375,0....|
|(125,[0,1,2,3,4,5...|[0.33333333333333...|
+--------------------+--------------------+
only showing top 5 rows



In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.regression import LinearRegression 

IP_train_data,IP_test_data = IP_scaler_df.randomSplit([0.8,0.2],seed=2)
IP_Regressor=LinearRegression(featuresCol ='scaledFeatures', labelCol ='selling_price') 
IP_Regressor=IP_Regressor.fit(IP_train_data)

IP_predicited=IP_Regressor.evaluate(IP_test_data)

coeff = IP_Regressor.coefficients

#X and Y intercept
intr = IP_Regressor.intercept

print("Using Feaure hasher to reduce the features space to dimension= 5 and \nPolynomial Regression using to expand by degree : 4 ")
print ("The coefficient of the model is : %a" %coeff)
print ("The Intercept of the model is : %f" %intr)

#Using evalaution mertics to determine the model performance
IP_eval = RegressionEvaluator(labelCol="selling_price", predictionCol="prediction", metricName="rmse")

rmse = IP_eval.evaluate(IP_predicited.predictions)
print("RMSE: %.3f" %rmse)
mse = IP_eval.evaluate(IP_predicited.predictions, {IP_eval.metricName: "mse"})
print("MSE: %.3f" % mse)

r2= IP_eval.evaluate(IP_predicited.predictions, {IP_eval.metricName: "r2"})
print("r2: %.3f" %r2)

Using Feaure hasher to reduce the features space to dimension= 5 and 
Polynomial Regression using to expand by degree : 4 
The coefficient of the model is : DenseVector([-41888417.3235, 93607087.2481, -64595103.1143, 12834314.5954, -1160571.073, 15205922.2716, -25340775.569, 8184871.6815, -7565231.4116, 8278166.3441, 1598537.0702, -2087242.225, -6776991.4508, 5169000.8761, 23747791.8471, -4775531.817, 4937401.1661, -2484890.5268, -24634208.4748, 5972253.268, 2700699.1209, -11905030.133, -5740078.9516, 21913594.5899, -71281947.2269, 1459925.5842, 1682245.6256, 30290606.533, -8189768.9894, 2105649.2869, 72971239.6524, -538903.8002, -13474172.0391, -23162540.4194, -54100961.0247, 69305170.769, -70504665.2021, 28944740.3668, 43707344.6349, -51914512.4944, 12177009.6903, 75978853.998, 2488758.1781, -60699333.2166, 17071456.8138, -2797085.5381, -8249922.6463, -10355432.6829, 23844438.7711, -45261910.8076, -26679892.4997, 11943145.9507, 1353346.062, -5992435.6069, 72321038.1628, -41191427.087

>The results in CASE-4 obtained using imputations on numeric columns with mising values
and applying Feature Hasher where the feature space is reduced to 5 on applying 
Polynomial Expansion by degree 4 followed by building a regression model for it gives best results among all the cases.